In [3]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
import json
from transformers import RobertaModel, RobertaTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

input_file_path = r"D:\Sharif University of Tech\Data\Library Recommender\Pypi data\OriginalItems.json"
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)


data_df = pd.DataFrame(data)

Lets draw one meaningfull representation 

In [4]:
data_df['text'] = data_df['Summary'].str[0] + " " + data_df['Description'].str[0]
data_df['text'] = data_df['text'].fillna('')

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Lets start embedding :

In [6]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

data_df['embedding'] = data_df['text'].apply(get_embedding)

And to find similar libraries :

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_libraries(query, data_df, top_n=5):
    query_embedding = get_embedding(query)
    similarities = []

    for _, row in data_df.iterrows():
        library_embedding = row['embedding']
        similarity = cosine_similarity(query_embedding, library_embedding)
        similarities.append((row['Package'][0], similarity[0][0]))

    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    return sorted_similarities[:top_n]

In [25]:
query = "I need a library for read and proccess data"

In [26]:
similar_libraries = find_similar_libraries(query, data_df)

In [27]:
similar_libraries

[('nsepy-v1 0.1', 0.98204875),
 ('printListModule 1.0.0', 0.98187697),
 ('pointstorm 1.1.9', 0.98175424),
 ('wyjtools 0.0.1', 0.98125494),
 ('takeme-client-python 2.9', 0.98088366)]

In [30]:
pandas_row = data_df[data_df['Package'].apply(lambda x: 'pandas' in x[0])]
pandas_description = pandas_row['Description'].iloc[0][0]

In [31]:
pandas_description

"pandas_multi ============ Simple loop for reading multiple csv files (matching a certain pattern) as a ``pandas.DataFrame``. I'm aware this need can be solved in even one line of Python, but loading multiple similar csv's is just something that should be as easy as loading one csv. If you don't want to add a new dependency to your project, google what ``os.listdir`` and ``glob`` can do for you. Installation can be done by typing:: pip install pandas_multi Usage of ``pandas_multi.read_csvs`` has been kept as similar as possible to ``pandas.read_csv``:: import pandas_multi # #   Note that dataframes only work if you give them the non-descriptive name df # df = pandas_multi.read_csvs('./20180728*.csv') # if you provide it with a path to a folder and nothing else, it will assume # everything in the folder is a comma-separated file df = pandas_multi.read_csvs('./data/') # if this is not the case, do this: df = pandas_multi.read_csvs('./data/*.csv') All options that are available to ``panda

This is theoreticly good, but functionally we could do much better

Lets start using NLP 

In [32]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [33]:
def preprocess(text):
    return text.lower()

def get_embedding(text):
    processed_text = preprocess(text)
    embedding = model.encode(processed_text, convert_to_tensor=True)
    return embedding.numpy()

from sklearn.metrics.pairwise import cosine_similarity

def find_similar_libraries(query, data_df, top_n=5):
    query_embedding = get_embedding(query)
    similarities = []

    for index, row in data_df.iterrows():
        library_embedding = row['embedding']
        similarity = cosine_similarity([query_embedding], [library_embedding])
        similarities.append((row['Package'][0], similarity[0][0]))

    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return sorted_similarities[:top_n]

In [34]:
data_df['embedding'] = data_df['text'].apply(get_embedding)

In [36]:
query = "csv"
similar_libraries = find_similar_libraries(query, data_df)
print(similar_libraries)

[('bankreport 0.1.6', 0.5945563), ('topsis-jatin-101703263 1.0', 0.5806109), ('only-common 2.1.0', 0.5434755), ('csvconvert 0.1', 0.53128356), ('bsdb 2.0.1', 0.5235471)]
